## CC3001 Otoño 2022 Tarea 3 [JUAN ESPINOZA C.]

## Diferencias entre archivos

### Profesores
Sección 1 Iván Sipirán •
Sección 2 Patricio Poblete •
Sección 3 Nelson Baloian



# Introducción

La _distancia de Levenshtein_, también llamada _distancia de edición_, es una función que indica la similitud que existe entre dos cadenas de texto. Informalmente, se puede decir que la distancia de Levenshtein mide la mínima cantidad de ediciones puntuales que hay que realizar sobre un _string_ para convertirlo en el otro.

Las operaciones permitidas son: inserción, borrado o sustitución de un caracter. Por lo tanto, una baja distancia de Levenshtein indica que los _strings_ son muy parecidos y una alta distancia indicará que son muy distintos. Matemáticamente, la distancia de Levenshtein entre dos strings $a$ y $b$ de largos $n$ y $m$ respectivamente, se denota como $\text{lev}_{a,b}(n, m)$ y se calcula recursivamente de la siguiente manera:

$$
\begin{align}
\text{lev}_{a,b}(i,0) &= i \\
\text{lev}_{a,b}(0,j) &= j \\
\text{lev}_{a,b}(i,j) &= \min\begin{cases}
\text{lev}_{a,b}(i-1,j)+1 \\
\text{lev}_{a,b}(i,j-1)+1 \\
\text{lev}_{a,b}(i-1,j-1)+\bf{1}_{(a_i \ne b_j)}
\end{cases} \text{para}~i,j>0
\end{align}
$$

En donde $\bf{1}_{(a_i \neq b_j)}$ es la función indicatriz, que toma el valor $1$ si $a_i \neq b_j$ y $0$ si no, y $\text{lev}_{a,b}(i,j)$ corresponde a la distancia entre los primeros $i$ caracteres de $a$ y los primeros $j$ de $b$. Notar que el
primer elemento en la función mínimo corresponde al caso en que debiera borrar el $i$-ésimo caracter de $a$, el segundo cuando hay que insertar el $j$-ésimo caracter de $b$ en $a$ y el tercero cuando el $i$-ésimo caracter de $a$ se debe reemplazar por el $j$-ésimo de $b$ (cuando son distintos), o si ese caracter se debe dejar tal cual está (cuando son iguales).

Por ejemplo, la distancia entre mojados y moteado es 3:

1. mojado**s** → mojado (eliminar s)
2. mo**j**ado → mo**t**ado (sustituir j por t)
3. motado → mot**e**ado (insertar e)

#Implementación 
Lo que se requiere que usted realice en esta tarea es una función similar al comando ``diff`` de Unix que muestra la diferencia entre dos archivos.
Para esto, usted debe crear su propia implementación de la función de distancia de Levenshtein, pero modificada para comparar línea por línea en lugar de caracter por caracter.
En base a esto, usted debe
encontrar cuáles líneas de un archivo $A$ deben borrarse, insertarse o sustituirse para que el resultado sea idéntico a
otro archivo $B$, usando la menor cantidad posible de operaciones.

Su solución a la tarea debe ser una función ``diff`` que genere y retorne una lista de items, que representan instrucciones de edición.Cada item puede ser una lista de largo 2 o 3, en el formato [$k$,’``E``’] para eliminar ($k$ es el número de línea del archivo de origen, comenzando de $1$ en adelante), [$k$,’``I``’,string] para insertar string a continuación de la linea $k$ y [$k$,’``S``’,string] para sustituir el contenido original de la linea $k$ por string.

Por ejemplo, si tenemos los siguientes archivos A.txt y B.txt

A.txt    | B.txt 
---------|------
Gato     | Gato 
Cerdo    | Tigre
Tigre    | Pato 
Pollo    | Perro
Perro    | Conejo
Caballo  | Caballo
Elefante |

La función deberia retornar la lista

```
[[2,'E'],
 [4,'S',"Pato"],
 [5,'I',"Conejo"],
 [7,'E']]
```


Como podrá imaginar, una solución que utilice fuerza bruta sería bastante lenta y, por lo tanto,
no es aceptable. Entonces, se requiere que usted implemente una versión que utilice un enfoque de
programación dinámica para resolver este problema. Particularmente se le pide que complete la función ``diff`` en el código a continuación para que retorne la lista indicada.

El código previo incluye una función ``lee_archivos`` que lee el contenido de dos archivos, los guarda línea por línea en una lista python, y otra función ``aplica_cambios``, que procesa la lista de cambios indicada sobre una lista que representa al archivo de entrada  y retorna la lista correspondiente al archivo de salida.

A continuación, para que vean cómo funciona, se aplica esto a los datos del ejemplo anterior.

En los recuadros siguientes está el código para procesar tres casos de prueba.

In [ ]:
#preparacion de ambiente para leer archivos ubicados en la internet
!wget /resources/data/A1.txt http://dcc.uchile.cl/nbaloian/cc3001/A1.txt
!wget /resources/data/B1.txt http://dcc.uchile.cl/nbaloian/cc3001/B1.txt
!wget /resources/data/A2.txt http://dcc.uchile.cl/nbaloian/cc3001/A2.txt
!wget /resources/data/B2.txt http://dcc.uchile.cl/nbaloian/cc3001/B2.txt
!wget /resources/data/A3.txt http://dcc.uchile.cl/nbaloian/cc3001/A3.txt
!wget /resources/data/B3.txt http://dcc.uchile.cl/nbaloian/cc3001/B3.txt
!wget /resources/data/A4.txt http://dcc.uchile.cl/nbaloian/cc3001/A4.txt
!wget /resources/data/B4.txt http://dcc.uchile.cl/nbaloian/cc3001/B4.txt


/resources/data/A1.txt: Scheme missing.
--2022-04-28 04:18:36--  http://dcc.uchile.cl/nbaloian/cc3001/A1.txt
Resolving dcc.uchile.cl (dcc.uchile.cl)... 192.80.24.11
Connecting to dcc.uchile.cl (dcc.uchile.cl)|192.80.24.11|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.dcc.uchile.cl/nbaloian/cc3001/A1.txt [following]
--2022-04-28 04:18:37--  https://www.dcc.uchile.cl/nbaloian/cc3001/A1.txt
Resolving www.dcc.uchile.cl (www.dcc.uchile.cl)... 200.9.99.213, 192.80.24.11
Connecting to www.dcc.uchile.cl (www.dcc.uchile.cl)|200.9.99.213|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://users.dcc.uchile.cl/~nbaloian/cc3001/A1.txt [following]
--2022-04-28 04:18:37--  https://users.dcc.uchile.cl/~nbaloian/cc3001/A1.txt
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 200.9.99.211, 192.80.24.4
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|200.9.99.211|:443... connected.
HTTP request sent, aw

In [ ]:
def lee_archivos(na,nb): #recibe dos nombres de archivos y los devuelve en una lista cada uno
  f = open(na,"r")
  a = []
  for linea in f:
    a.append(linea.rstrip())
  f = open(nb,"r")
  b = []
  for linea in f:
    b.append(linea.rstrip())
  return [a,b]

[a,b] = lee_archivos("A1.txt","B1.txt")
print(a)
print(b)

def aplica_cambios(A,Lista):
    L=Lista.copy()
    L.append([len(A)+1,'F']) # seudo-comando FIN
    j=1 # próxima línea a consumir del archivo de entrada
    B=[] # archivo de salida
    i=0 # ítem actual
    while j<=len(A) and i<len(L):
      k=L[i][0]
      assert j<=k
      if j<k: # líneas que no aparecen mencionadas en los comandos se copian
        B.append(A[j-1])
      else: # j==k, el comando se refiere a esta línea
        if L[i][1]=="E":
          pass # línea a eliminar, se ignora
        elif L[i][1]=="S":
          B.append(L[i][2])
        elif L[i][1]=="I":
          B.append(A[j-1])
          B.append(L[i][2])
        # procesamos otras operaciones sobre la misma línea, en caso de haberlas
        i+=1
        while j==L[i][0]:
          if L[i][1]=="I": # ignoramos cualquier cosa que no sea "I"
            B.append(L[i][2])
          i+=1
      # finalmente avanzamos a la siguiente línea
      j+=1
    return B
import numpy as np
def diff(a,b):
    m=len(a) #se comienza relizando la matriz de la distancia de levenshtein
    n=len(b)
    E=[]
    L=np.zeros((m+1,n+1),dtype=int) 
    for i in range(0,m+1):
        L[i,0]=i
    for j in range(0,n+1):
        L[0,j]=j
    for i in range(1,m+1):
        for j in range(1,n+1):
            x=L[i-1,j]+1
            y=L[i,j-1]+1
            k=0
            if a[i-1]!=b[j-1]:
                k=1
            z=L[i-1,j-1]+k
            L[i,j]=min(x,y,z)   
    def read(L,i,j,a,b): #luego, una funcion que lea la matriz desde su ultimo elemento (inferior derecho), el cual es la distancia min., buscando los minimos de sus vecinos (diagonal, arriba, izquierda) hasta llegar al [0,0]
        if (i,j)==(0,0):
            return E
        if i==0: #se encuentra en el borde izquierdo de la matriz, solo le queda subir a [0,0]
            E.append([i,'I',b[j-1]])
            return read(L,i,j-1,a,b)
        if j==0: #se encuentra en el borde superior de la matriz, solo le queda moverse hacia la izquierda a [0,0]
            E.append([i,'E'])
            return read(L,i-1,j,a,b)
        x=L[i-1,j] #arriba
        y=L[i,j-1] #izquierda
        z=L[i-1,j-1] #diagonal
        if i==len(a):
            if min(x,y,z)==x:
                E.append([i,'E'])
                return read(L,i-1,j,a,b)
        if min(x,y,z)==z:
            if a[i-1]==b[j-1]: #si son iguales no hay sustitucion
                return read(L,i-1,j-1,a,b)
            else: #no son iguales -> hay sustitucion
                E.append([i,'S',b[j-1]])
                return read(L,i-1,j-1,a,b)
        if min(x,y,z)==x: #hacia arriba -> eliminar 
            E.append([i,'E'])
            return read(L,i-1,j,a,b)
        if min(x,y,z)==y: #hacia la izquierda -> insertar 
            E.append([i,'I',b[j-1]])
            return read(L,i,j-1,a,b)
    read(L,len(a),len(b),a,b)
    E.reverse() #la lista me quedaba invertida :c
    return E


  

['este es un', 'ejemplo de', 'archivo que', 'se compara', 'con uno parecido']
['este es un', 'pequeño pero significativo', 'ejemplo de', 'archivo que', 'se compara', 'con uno parecido', 'pero tiene', 'mas lineas']


A continuación, probamos "a mano" el caso del ejemplo:

In [ ]:
A=['Gato', 'Cerdo', 'Tigre', 'Pollo', 'Perro', 'Caballo', 'Elefante']
B=['Gato', 'Tigre', 'Pato', 'Perro', 'Conejo', 'Caballo']
L=[[2, 'E'], [4, 'S', 'Pato'], [5, 'I', 'Conejo'], [7, 'E']]
C=aplica_cambios(A,L)
print("Al aplicar los cambios sobre A el resultado es:")
print(C)
print("OK" if B==C else "Error")

Al aplicar los cambios sobre A el resultado es:
['Gato', 'Tigre', 'Pato', 'Perro', 'Conejo', 'Caballo']
OK


Para chequear si su función hace lo que requiere use los siguientes chequeos. Estos chequeos también determinan si tu solución tiene el número óptimo de operaciones.



In [ ]:
def chequeo(arch1,arch2, optimo):
  [A,B] = lee_archivos(arch1,arch2)
  L = diff(A,B)
  C=aplica_cambios(A,L)
  print("Al aplicar los cambios sobre A el resultado es:")
  print(C)
  print("OK" if B==C and len(L)==optimo else "Error")

chequeo("A1.txt","B1.txt", 3)
chequeo("A2.txt","B2.txt", 2)
chequeo("A3.txt","B3.txt", 2)
chequeo("A4.txt","B4.txt", 4)

Al aplicar los cambios sobre A el resultado es:
['este es un', 'pequeño pero significativo', 'ejemplo de', 'archivo que', 'se compara', 'con uno parecido', 'pero tiene', 'mas lineas']
OK
Al aplicar los cambios sobre A el resultado es:
['Pato', 'Perro', 'Conejo']
OK
Al aplicar los cambios sobre A el resultado es:
['1', '2', '3', '4', '5', '6', '7', '8', '9']
OK
Al aplicar los cambios sobre A el resultado es:
['Gato', 'Tigre', 'Pato', 'Perro', 'Conejo', 'Caballo']
OK


## ¿Qué hay que entregar?

Usted debe crear una copia de este documento (del menú seleccionar "Archivo" o "File" y de ahi la opción "guardar copia en Drive" o "save a copy in Drive"). Este nuevo documento (notebook de Colab) es el que debe modificar para luego compartirlo con la opción "Cualquier usuario de internet con este enlace puede ver esto" (o “Anyone on the internet with this link can view”). Esta opción se encuentra disponible en la parte superior derecha de su notebook, en el botón "Compartir" (o "Share"). Una vez que tenga disponible el enlace, colocarlo en la descripción de la entrega de su tarea. No olvide colocar su nombre dentro del notebook.